In [1]:
import tensorflow as tf
import numpy as np
import cPickle
%matplotlib inline
import matplotlib.pyplot as plt

import dataset_class as dataset

In [2]:
# Constants
data_dir = "./data/cifar-10-batches-py/"
logs_path = "/tmp/tensorflow/log"    

if tf.gfile.Exists(logs_path):
    tf.gfile.DeleteRecursively(logs_path)
tf.gfile.MakeDirs(logs_path)

tf.reset_default_graph()

In [3]:
# Hyper parameter
n_classes = 10
batch_size = 128
image_width = 32
image_height = 32
image_depth = 3
learning_rate = 0.001
epochs = 2500

In [4]:
def unpickle(file):
    
    fo = open(file, 'rb')
    dict = cPickle.load(fo)
    fo.close()
    return dict

In [5]:
def loadBatch(batch_number):
    return unpickle(data_dir + "data_batch_" + str(batch_number))

def loadTest():
    return unpickle(data_dir + "test_batch")

def getData():
    Xlist, ylist=[], []
    
    for i in range(1, 6):
        batch = loadBatch(i)
        Xlist.append(batch['data'])
        ylist.append(batch['labels'])
        
    

    
    test = loadTest()
    
    y_test = np.vstack(test['labels'])   
    y_test = np.reshape(y_test,(-1,1))
    one_hot_test = np.zeros((y_test.shape[0],10),dtype="int64")
    
    
    for i in xrange(y_test.shape[0]):
        one_hot_test[i,y_test[i]]=1
    
    y_test=one_hot_test.astype("float32")
    
    
    X_test = np.vstack(test['data'])
    X_test = X_test.reshape((X_test.shape[0],3,32,32)).transpose((0,2,3,1)) 

    
    y_train = np.vstack(ylist)   
    y_train = np.reshape(y_train,(-1,1))
    one_hot_train=np.zeros((y_train.shape[0],10),dtype="int64")      
    
    for i in xrange(y_train.shape[0]):
        one_hot_train[i,y_train[i]]=1
        
    y_train=one_hot_train.astype("float32")
    

    X_train = np.vstack(Xlist)
    X_train = X_train.reshape((X_train.shape[0],3,32,32)).transpose((0,2,3,1))



    
    return dataset.read_data_sets(X_train,y_train, X_test, y_test, None, False)

def getDisortedBatch(batch):    
    sess = tf.Session()
    
    for i in range(0, X_train.shape[0]):
        tf.random


    return batch



In [6]:
# Helper functions
def weight_variable(shape, varName):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial, name=varName)

def bias_variable(shape, varName):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial, name=varName)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [7]:
def distortInput(images):
    return imagess

In [8]:
# Placeholders

with tf.name_scope("input"):
    x = tf.placeholder(dtype=tf.float32, shape=[None, 32, 32, 3], name="x-input")
    y_ = tf.placeholder(dtype=tf.float32, shape=[None, 10], name="y-input")
    #x = tf.image.random_brightness(x, 0.5)
    #x = tf.image.flip_left_right(x)
    #x = tf.image.random_hue(x, 0.3)

    
with tf.name_scope("conv1"):
    W_conv1 = weight_variable([3, 3, 3, 64], "weights")
    b_conv1 = bias_variable([64], "biases")

    h_conv1 = tf.nn.relu(conv2d(x , W_conv1) + b_conv1)
    conv1_pool = max_pool_2x2(h_conv1)
    
    conv1_out = tf.nn.lrn(conv1_pool, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,
                    name='norm1')
    print "conv1_out shape: ", conv1_out.get_shape()


with tf.name_scope("conv2"):
    W_conv2 = weight_variable([3, 3, 64, 64], "weights")
    b_conv2 = bias_variable([64], "biases")

    h_conv2 = tf.nn.relu(conv2d(conv1_out , W_conv2) + b_conv2)
    conv2_norm = tf.nn.lrn(h_conv2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,
                    name='norm2')
    conv2_out = max_pool_2x2(conv2_norm)
    print "conv2_out shape:", conv2_out.get_shape()
    

with tf.name_scope("conv3"):
    W_conv3 = weight_variable([3, 3, 64, 64], "weights")
    b_conv3 = bias_variable([64], "biases")

    h_conv3 = tf.nn.relu(conv2d(conv2_out , W_conv3) + b_conv3)
    conv3_norm = tf.nn.lrn(h_conv3, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,
                    name='norm2')
    
    print "conv2_out shape:", conv2_out.get_shape()

with tf.name_scope("fc1"):
    W_fc1 = weight_variable([8 * 8 * 64, 256], "weights")
    b_fc1 = bias_variable([256], "biases")

    conv2_flat = tf.reshape(conv3_norm, [-1, 8 * 8 * 64])
    fc1_out = tf.nn.relu(tf.matmul(conv2_flat, W_fc1) + b_fc1)
    print "fc1_out shape:", fc1_out.get_shape()

with tf.name_scope("dropout"):
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(fc1_out, keep_prob)

with tf.name_scope("fc2"):
    W_fc2 = weight_variable([256, 128], "weights")
    b_fc2 = bias_variable([128], "biases")

    fc2_out = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
    #fc2_out = tf.nn.relu(tf.matmul(fc1_out, W_fc2) + b_fc2)
    print "fc2_out shape:", fc2_out.get_shape()

with tf.name_scope("output"):
    W_fc3 = weight_variable([128, 10], "weights")
    b_fc3 = bias_variable([10], "biases")

    y_conv = tf.matmul(fc2_out, W_fc3) + b_fc3
    print "y_conv shape:", y_conv.get_shape()
    print "y_ shape:", y_.get_shape()

with tf.name_scope("cross_entropy"):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_conv, y_))

with tf.name_scope("train"):
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

with tf.name_scope("accuracy"):
    correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    
#create a summary for our cost and accuracy
tf.scalar_summary("cost", cross_entropy)
tf.scalar_summary("accuracy", accuracy)


summary_op = tf.merge_all_summaries()

    

conv1_out shape:  (?, 16, 16, 64)
conv2_out shape: (?, 8, 8, 64)
conv2_out shape: (?, 8, 8, 64)
fc1_out shape: (?, 256)
fc2_out shape: (?, 128)
y_conv shape: (?, 10)
y_ shape: (?, 10)


In [9]:

def train():
    
    sess = tf.Session()
    
    train_writer = tf.train.SummaryWriter(logs_path + "/train", sess.graph)
    test_writer = tf.train.SummaryWriter(logs_path + "/test")

    #sess.initialize_all_variables().run()
    sess.run(tf.global_variables_initializer())
    
    batch_1 = loadBatch(1)
    
    CIFAR10 = getData()


    for i in range(epochs):
        batch = CIFAR10.train.next_batch(batch_size)     
        if (i % 10 == 0):
            summ, acc = sess.run([summary_op, accuracy], feed_dict={x: batch[0], y_: batch[1],  keep_prob: 1.0})

            train_writer.add_summary(summ, i)

            print "Epoch: " , i , " Acc: ", acc
        train_step.run(session=sess, feed_dict={x:batch[0],y_:batch[1],  keep_prob: 0.5})
       
    
    FINAL_ACC=0.
    for i in xrange(0,10):
        FINAL_ACC+=0.1*sess.run(accuracy, feed_dict={x: CIFAR10.test.images[i*1000:(i+1)*1000], y_: CIFAR10.test.labels[i*1000:(i+1)*1000]}) 


    print "Final accuracy on test set:", FINAL_ACC    

    train_writer.close()
    test_writer.close()

In [10]:
train()

Epoch:  0  Acc:  0.15625
Epoch:  10  Acc:  0.140625
Epoch:  20  Acc:  0.132812
Epoch:  30  Acc:  0.117188
Epoch:  40  Acc:  0.0546875
Epoch:  50  Acc:  0.0703125
Epoch:  60  Acc:  0.0859375
Epoch:  70  Acc:  0.09375
Epoch:  80  Acc:  0.125
Epoch:  90  Acc:  0.0859375
Epoch:  100  Acc:  0.117188
Epoch:  110  Acc:  0.101562
Epoch:  120  Acc:  0.0859375
Epoch:  130  Acc:  0.148438
Epoch:  140  Acc:  0.0703125
Epoch:  150  Acc:  0.171875
Epoch:  160  Acc:  0.078125
Epoch:  170  Acc:  0.109375
Epoch:  180  Acc:  0.109375
Epoch:  190  Acc:  0.109375
Epoch:  200  Acc:  0.117188
Epoch:  210  Acc:  0.0859375
Epoch:  220  Acc:  0.140625
Epoch:  230  Acc:  0.09375
Epoch:  240  Acc:  0.0859375
Epoch:  250  Acc:  0.09375
Epoch:  260  Acc:  0.125
Epoch:  270  Acc:  0.0546875
Epoch:  280  Acc:  0.109375
Epoch:  290  Acc:  0.117188
Epoch:  300  Acc:  0.101562
Epoch:  310  Acc:  0.132812
Epoch:  320  Acc:  0.101562
Epoch:  330  Acc:  0.09375
Epoch:  340  Acc:  0.148438
Epoch:  350  Acc:  0.117188
Epoch

InvalidArgumentError: You must feed a value for placeholder tensor 'dropout/Placeholder' with dtype float
	 [[Node: dropout/Placeholder = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'dropout/Placeholder', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/jojo/.local/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/jojo/.local/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/jojo/.local/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/home/jojo/.local/lib/python2.7/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self

In [ ]:
batch_1 = loadBatch(1)
    
y_train = np.vstack(batch_1['labels'])   
y_train = np.reshape(y_train,(-1,1))
one_hot_train=np.zeros((y_train.shape[0],10),dtype="int64")

X_test = np.vstack(batch_1['data'])
X_test = X_test.reshape((X_test.shape[0],3,32,32)).transpose((0,2,3,1)) 
X_test[0].shape
img = X_test[4]


img = tf.random_crop(img, [image_height, image_width, 3])

sess = tf.Session()
#plt.imshow(img)
with sess.as_default():
    print img.eval()

In [ ]:
tf.image.random_brightness?